# Description

This notebook creates a deep neural network model (MLP) to predict future returns based on fundamental factors.

I got the data from the SimFin free tier. I believe they give about 5 years of historical data for US
stocks and it is not quite recent

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
# Import the main functionality from the SimFin Python API.
import simfin as sf
# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

In [2]:
# SimFin data-directory.
sf.set_data_dir('~/.simfin/data/')
# SimFin load API key or use free data.
sf.load_api_key(path='~/.simfin/api-key.txt', default_key='free')

In [3]:
# Seaborn set plotting style.
sns.set_style("whitegrid")

## Load Data

In [ ]:
%%time
# Data for USA.
market = 'us'
# TTM Income Statements.
df_income_ttm = sf.load_income(variant='ttm', market=market)
df_income_qrt = sf.load_income(variant='quarterly', market=market)
# Quarterly Income Statements.
df_balance_ttm = sf.load_balance(variant='ttm', market=market)
df_balance_qrt = sf.load_balance(variant='quarterly', market=market)
# Quarterly Balance Sheets.
df_cashflow_ttm = sf.load_cashflow(variant='ttm', market=market)
df_cashflow_qrt = sf.load_cashflow(variant='quarterly', market=market)
# Daily Share-Prices.
df_prices = sf.load_shareprices(variant='daily', market=market)

In [5]:
p_ticks = set(df_prices.index.get_level_values(0))
i1_ticks = set(df_income_ttm.index.get_level_values(0))
i2_ticks = set(df_income_qrt.index.get_level_values(0))
b1_ticks = set(df_balance_ttm.index.get_level_values(0))
b2_ticks = set(df_balance_qrt.index.get_level_values(0))
c1_ticks = set(df_cashflow_ttm.index.get_level_values(0))
c2_ticks = set(df_cashflow_qrt.index.get_level_values(0))

all_ticks = p_ticks.intersection(i1_ticks, i2_ticks, b1_ticks, b2_ticks, c1_ticks, c2_ticks)
tickers = list(all_ticks)
len(tickers)

3656

In [6]:
#tickers = ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'META', 'GOOG', 'LLY', 'AVGO', 'TSLA', 'UNH']

df_income_ttm = df_income_ttm.loc[tickers].copy()
df_income_qrt = df_income_qrt.loc[tickers].copy()
df_balance_ttm = df_balance_ttm.loc[tickers].copy()
df_balance_qrt = df_balance_qrt.loc[tickers].copy()
df_cashflow_ttm = df_cashflow_ttm.loc[tickers].copy()
df_cashflow_qrt = df_cashflow_qrt.loc[tickers].copy()
df_prices = df_prices.loc[tickers].copy()

## FIN Singals

In [7]:
df_income_ttm.shape, df_balance_ttm.shape, df_cashflow_ttm.shape, df_prices.shape

((56365, 26), (56362, 28), (56362, 26), (3763011, 9))

In [ ]:
df_fin_signals = sf.fin_signals(df_prices=df_prices,
                                df_income_ttm=df_income_ttm,
                                df_balance_ttm=df_balance_ttm,
                                df_cashflow_ttm=df_cashflow_ttm,
                                fill_method='ffill')

df_fin_signals.dropna().head()

## VAL Signals

In [9]:
df_val_signals = sf.val_signals(df_prices=df_prices,
                                df_income_ttm=df_income_ttm,
                                df_balance_ttm=df_balance_ttm,
                                df_cashflow_ttm=df_cashflow_ttm)
df_val_signals.dropna().head()

Dividend Yield  Earnings Yield  FCF Yield    Market-Cap  \
Ticker Date                                                                  
JW-A   2018-10-31        0.023773        0.061271   0.051419  3.149622e+09   
       2018-11-01        0.023067        0.059452   0.049892  3.246015e+09   
       2018-11-02        0.023084        0.059494   0.049927  3.243692e+09   
       2018-11-05        0.023167        0.059708   0.050107  3.232079e+09   
       2018-11-06        0.023022        0.059335   0.049794  3.252403e+09   

                      P/Cash        P/E      P/FCF    P/NCAV  P/NetNet  \
Ticker Date                                                              
JW-A   2018-10-31  27.245157  16.320891  19.448233 -2.964149 -2.622197   
       2018-11-01  28.078987  16.820387  20.043441 -3.054866 -2.702448   
       2018-11-02  28.058895  16.808351  20.029098 -3.052680 -2.700514   
       2018-11-05  27.958434  16.748171  19.957387 -3.041750 -2.690845   
       2018-11-06  28.134241  16.853487  20.082882 -3.060877 -2.707766   

                    P/Sales  Price to Book Value  
Ticker Date                                       
JW-A   2018-10-31  1.757159             2.693384  
       2018-11-01  1.810937             2.775814  
       2018-11-02  1.809641             2.773828  
       2018-11-05  1.803162             2.763897  
       2018-11-06  1.814500             2.781277

## Growth Signals

In [10]:
df_growth_signals = \
    sf.growth_signals(df_prices=df_prices,
                      df_income_ttm=df_income_ttm,
                      df_income_qrt=df_income_qrt,
                      df_balance_ttm=df_balance_ttm,
                      df_balance_qrt=df_balance_qrt,
                      df_cashflow_ttm=df_cashflow_ttm,
                      df_cashflow_qrt=df_cashflow_qrt)

df_growth_signals.dropna().head()

Assets Growth  Assets Growth QOQ  Assets Growth YOY  \
Ticker Date                                                              
EAR    2021-09-30       1.280529           -0.06797           1.280529   
       2021-10-01       1.280529           -0.06797           1.280529   
       2021-10-04       1.280529           -0.06797           1.280529   
       2021-10-05       1.280529           -0.06797           1.280529   
       2021-10-06       1.280529           -0.06797           1.280529   

                   Earnings Growth  Earnings Growth QOQ  Earnings Growth YOY  \
Ticker Date                                                                    
EAR    2021-09-30         1.996329             3.104906             7.154946   
       2021-10-01         1.996329             3.104906             7.154946   
       2021-10-04         1.996329             3.104906             7.154946   
       2021-10-05         1.996329             3.104906             7.154946   
       2021-10-06         1.996329             3.104906             7.154946   

                   FCF Growth  FCF Growth QOQ  FCF Growth YOY  Sales Growth  \
Ticker Date                                                                   
EAR    2021-09-30    0.944264       -0.004117         3.83279     -0.225663   
       2021-10-01    0.944264       -0.004117         3.83279     -0.225663   
       2021-10-04    0.944264       -0.004117         3.83279     -0.225663   
       2021-10-05    0.944264       -0.004117         3.83279     -0.225663   
       2021-10-06    0.944264       -0.004117         3.83279     -0.225663   

                   Sales Growth QOQ  Sales Growth YOY  
Ticker Date                                            
EAR    2021-09-30         -1.999388         -2.257506  
       2021-10-01         -1.999388         -2.257506  
       2021-10-04         -1.999388         -2.257506  
       2021-10-05         -1.999388         -2.257506  
       2021-10-06         -1.999388         -2.257506

## Create DF of factors we want

In [71]:
df_factors = pd.DataFrame(index=df_prices.index)

In [72]:
df_factors[CLOSE] = df_prices[ADJ_CLOSE]

In [73]:
df_factors.columns

Index(['Close'], dtype='object')

In [76]:
df_factors.query('Close > 10000')

Close
Ticker Date               
GRMM   2021-05-19  11700.0
TTOO   2018-08-28  31000.0
       2018-08-29  32150.0
       2018-08-30  31800.0
       2018-08-31  32650.0
...                    ...
SINT   2022-08-16  11000.0
       2022-08-17  11008.0
       2022-08-18  11002.0
       2022-08-19  10710.0
       2022-08-22  10574.0

[3377 rows x 1 columns]

In [63]:
df_factors["Return 3"] = sf.rel_change(df=df_prices[ADJ_CLOSE], freq='bdays', bdays=63, future=True)
df_factors["Return 6"] = sf.rel_change(df=df_prices[ADJ_CLOSE], freq='bdays', bdays=126, future=True)
df_factors["Return 12"] = sf.rel_change(df=df_prices[ADJ_CLOSE], freq='bdays', bdays=252, future=True)

In [64]:
df_factors.describe()

/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Adj. Close,Return 3,Return 6,Return 12
count,3.763011e+06,3.528574e+06,3.301948e+06,2.852496e+06
mean,2.348637e+02,inf,inf,inf
std,7.990159e+03,NaN,NaN,NaN
min,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,8.810000e+00,-1.496075e-01,-2.172414e-01,-3.153406e-01
50%,2.256000e+01,0.000000e+00,0.000000e+00,-2.907093e-03
75%,5.770000e+01,1.431759e-01,2.120323e-01,3.359972e-01
max,1.587600e+06,inf,inf,inf


In [56]:
df_factors.loc['AMZN'].head()

,Adj. Close,Return 3,Return 6,Return 12
Date,,,,
2018-08-28,96.64,-0.181809,-0.135037,-0.075745
2018-08-29,99.91,-0.160344,-0.151136,-0.111100
2018-08-30,100.12,-0.164203,-0.154814,-0.106173
2018-08-31,100.64,-0.160274,-0.170807,-0.105425
2018-09-04,101.98,-0.131006,-0.202785,-0.097470


In [58]:
df_growth_wins = sf.winsorize(df=df_growth_signals, quantile=0.01)
df_fin_wins = sf.winsorize(df=df_fin_signals, quantile=0.01)
df_value_wins = sf.winsorize(df=df_val_signals, quantile=0.01)

In [59]:
df_factors[EARNINGS_GROWTH_QOQ] =  df_growth_wins[EARNINGS_GROWTH_QOQ]
df_factors[SALES_GROWTH_QOQ] =  df_growth_wins[SALES_GROWTH_QOQ]

df_factors[CURRENT_RATIO] =  df_fin_wins[CURRENT_RATIO]
df_factors[DEBT_RATIO] =  df_fin_wins[DEBT_RATIO]
df_factors[GROSS_PROFIT_MARGIN] =  df_fin_wins[GROSS_PROFIT_MARGIN]
df_factors[NET_PROFIT_MARGIN] =  df_fin_wins[NET_PROFIT_MARGIN]
df_factors[BUYBACK_RATIO] =  df_fin_wins[BUYBACK_RATIO]
df_factors[ROA] =  df_fin_wins[ROA]
df_factors[ROE] =  df_fin_wins[ROE]

df_factors[MARKET_CAP] =  df_value_wins[MARKET_CAP]
df_factors[PE] =  df_value_wins[PE]
df_factors[PFCF] =  df_value_wins[PFCF]
df_factors[PSALES] =  df_value_wins[PSALES]
df_factors[PRICE_BOOK] =  df_value_wins[PRICE_BOOK]

In [60]:
df_factors.dropna().describe()

/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Adj. Close,Return 3,Return 6,Return 12,Earnings Growth QOQ,Sales Growth QOQ,Current Ratio,Debt Ratio,Gross Profit Margin,Net Profit Margin,Share Buyback / FCF,Return on Assets,Return on Equity,Market-Cap,P/E,P/FCF,P/Sales,Price to Book Value
count,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06
mean,1.007676e+02,inf,inf,inf,-9.076301e-02,7.979520e-02,2.790026e+00,3.445988e-01,4.187460e-01,-1.519241e+00,3.247572e-01,-3.598612e-02,-3.954617e-02,1.254687e+10,8.410928e+00,1.016872e+01,1.528084e+01,4.635457e+00
std,1.981251e+03,NaN,NaN,NaN,4.106851e+00,5.244952e-01,3.610800e+00,2.536205e-01,2.682493e-01,1.250942e+01,2.706309e+00,2.525483e-01,1.103931e+00,3.120637e+10,9.904710e+01,1.023578e+02,9.941590e+01,1.371650e+01
min,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-2.010808e+01,-1.000000e+00,8.921519e-02,2.297289e-03,-6.670599e-01,-1.571427e+02,-1.339018e+01,-2.088311e+00,-7.105951e+00,1.807876e+06,-5.620784e+02,-6.293041e+02,1.641182e-02,-5.586687e+01
25%,1.203000e+01,-1.306818e-01,-1.937830e-01,-2.605695e-01,-7.058236e-01,-5.631298e-02,1.176578e+00,1.599819e-01,2.330561e-01,-8.295750e-02,0.000000e+00,-4.882859e-02,-1.101392e-01,4.312118e+08,-5.115358e+00,-3.295886e+00,7.940779e-01,1.270087e+00
50%,3.084000e+01,1.064442e-02,1.342967e-02,2.582101e-02,-9.512293e-02,2.522640e-02,1.811017e+00,3.165685e-01,3.945771e-01,3.610348e-02,4.112508e-02,2.475692e-02,6.890450e-02,2.100868e+09,1.232337e+01,1.073135e+01,1.962195e+00,2.514423e+00
75%,7.236000e+01,1.540825e-01,2.317499e-01,3.663089e-01,3.453966e-01,1.137080e-01,2.948299e+00,4.690246e-01,5.985039e-01,1.053069e-01,5.099201e-01,6.845488e-02,1.824793e-01,8.666620e+09,2.839604e+01,2.525739e+01,5.061148e+00,5.218312e+00
max,4.394250e+05,inf,inf,inf,2.190000e+01,4.857182e+00,3.667269e+01,1.507345e+00,9.991566e-01,8.920483e-01,1.585466e+01,3.536072e-01,5.424555e+00,2.012404e+11,4.315663e+02,4.799820e+02,1.156070e+03,9.149198e+01


In [49]:
df_factors.dropna().describe()

/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/marcusprewarski/projects/venv/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Us

,Adj. Close,Return 3,Return 6,Return 12,Earnings Growth QOQ,Sales Growth QOQ,Current Ratio,Debt Ratio,Gross Profit Margin,Net Profit Margin,Share Buyback / FCF,Return on Assets,Return on Equity,Market-Cap,P/E,P/FCF,P/Sales,Price to Book Value
count,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06,1.740477e+06
mean,1.007676e+02,inf,inf,inf,NaN,NaN,inf,inf,NaN,NaN,3.566935e-01,-inf,-4.402636e+00,1.590904e+10,inf,7.486617e+02,inf,1.293140e+02
std,1.981251e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.726804e+01,NaN,7.203007e+02,7.871128e+10,NaN,2.579383e+05,NaN,1.957020e+04
min,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-inf,-inf,0.000000e+00,0.000000e+00,-inf,-inf,-3.582970e+03,-inf,-1.187739e+05,0.000000e+00,-2.840574e+06,-4.390967e+07,-1.137265e+04,-3.471836e+05
25%,1.203000e+01,-1.306818e-01,-1.937830e-01,-2.605695e-01,-7.058236e-01,-5.631298e-02,1.176578e+00,1.599819e-01,2.330561e-01,-8.295750e-02,0.000000e+00,-4.882859e-02,-1.101392e-01,4.312118e+08,-5.115358e+00,-3.295886e+00,7.940779e-01,1.270087e+00
50%,3.084000e+01,1.064442e-02,1.342967e-02,2.582101e-02,-9.512293e-02,2.522640e-02,1.811017e+00,3.165685e-01,3.945771e-01,3.610348e-02,4.112508e-02,2.475692e-02,6.890450e-02,2.100868e+09,1.232337e+01,1.073135e+01,1.962195e+00,2.514423e+00
75%,7.236000e+01,1.540825e-01,2.317499e-01,3.663089e-01,3.453966e-01,1.137080e-01,2.948299e+00,4.690246e-01,5.985039e-01,1.053069e-01,5.099201e-01,6.845488e-02,1.824793e-01,8.666620e+09,2.839604e+01,2.525739e+01,5.061148e+00,5.218312e+00
max,4.394250e+05,inf,inf,inf,inf,inf,inf,inf,inf,inf,4.568000e+03,1.163626e+02,8.375134e+02,3.048174e+12,inf,3.942020e+07,inf,3.565793e+06


## Add adjusted close

In [ ]:
factors_df[ADJ_CLOSE] = df_prices[ADJ_CLOSE]
factors_df

## Add 1 year return

In [ ]:
factors_df["One Year Return"] = sf.rel_change(df=df_prices[ADJ_CLOSE], freq='bdays', bdays=252, future=True)

## Stock Selection from our Factors

In [ ]:
def select_tickers(factors_df, max_tickers):
    select_df = factors_df.copy()
    # set negative EV/EBIT values to something high since we are ranking based on low values
    select_df['EV/EBIT'] = select_df['EV/EBIT'].mask(select_df['EV/EBIT'] < 0, 1000)
    # rank by EV/EVIT
    select_df = select_df.sort_values("EV/EBIT", ascending=False)
    select_df['Rank EV/EBIT'] = np.arange(select_df.shape[0])
    # rank by ROA
    select_df = select_df.sort_values("ROA", ascending=True)
    select_df['Rank ROA'] = np.arange(select_df.shape[0])
    # rank combined
    select_df["Rank Overall"] = select_df["Rank EV/EBIT"] + select_df["Rank ROA"]
    select_df = select_df.sort_values("Rank Overall", ascending=False)
    return select_df[0: max_tickers]

In [ ]:
date_df = factors_df.xs('2019-01-02', level='Date', drop_level=False)
sel_df = select_tickers(date_df, 25).dropna()
sel_df


In [ ]:
sel_df["One Year Return"].mean()

In [ ]:
date_df["One Year Return"].mean()

In [ ]:
ts = pd.Timestamp(2019, 1, 2)

for i in range(3):
    date_df = factors_df.xs(ts, level='Date', drop_level=False)
    sel_df = select_tickers(date_df, 25)
    sel_df = sel_df.dropna()
    print(f"sel25 return {sel_df['One Year Return'].mean()}  all return {date_df['One Year Return'].mean()}")
    ts = ts + pd.offsets.BusinessDay(n=252)

# Conclusion
These results are somewhat disappointing. I'm using ROI instead of ROIC but otherwise I think this is a fairly
similar model to the Magic Formula. There are only three years worth of data, but the selected stocks underperformed
the universe of stocks in each year and also the S&P 500. There could be errors in the calculations or data.